## TP4 循环神经网络  
### LSTM、GRU 和其他记忆单元

在本次实验中，我们将继续研究循环神经网络 (RNN) 及其在自然语言处理中的应用，着重研究两个问题：
1. 如何处理并生成可变长度的序列？
2. 如何处理更长时间的依赖关系（例如，消失/爆炸梯度问题）？

我们将重新审视实验3中的目标（生成序列）以及相同的数据集（特朗普的竞选前演讲）。因此，你需要将实验3中的代码复制到文件 `tp4.py` 中，并逐步进行修改，以回答不同的问题。

## 1  生成可变长度的序列

与上一节实验不同，这次我们将每个序列视为一个句子（可以以句号、问号或感叹号结尾）。因此，我们不再假设序列具有固定长度。为此，需要：

- 使用一个特殊符号来标记序列的结束（EOS）。在训练时，需要将该标记添加到每个序列末尾，以便模型学会预测它；
- 对每个序列进行填充，即根据需要添加空字符（BLANK），以确保同一批次中的所有序列具有相同的长度；在训练过程中应忽略此字符（参见问题2）。

### 问题 1

在 `textloader.py` 文件中，`TextDataset` 类具有以下特点：
- 每个样本是一句话（以“.”作为分隔符）；
- 数据集的大小等于语料库中的句子数量；
- 返回的句子为一系列整数，每个整数代表一个字符（应用 `string2code` 函数）。

该类返回长度不定的样本。因此，需要向 `DataLoader` 指定如何将这些样本组合成一个批次。这由 `DataLoader` 构造函数的 `collate_fn` 参数来实现，该参数接受一个函数作为参数。

定义一个函数 `pad_collate_fn`，将 `TextDataset` 的样本列表转换为批次张量（形状为 `最大长度 x 批次大小`，其中 `最大长度` 是批次中最长序列的长度）。该函数需要在每个样本的末尾添加 EOS 符号的编码，并用空字符的编码填充序列。运行 `textloader.py` 以确保一切正常。

此外，在计算损失（这里指最大似然）时，需要注意不要包含填充部分。为此，最常用的解决方法是使用一个二值掩码（填充字符对应0，其他字符对应1），在求和之前将该掩码与对数概率相乘（在损失函数中使用 `reduce="none"` 参数，尤其是在交叉熵损失函数中，可以得到每个元素的损失值，而不是平均值）。

### 问题 2

在 `tp4.py` 中创建损失函数 `maskedCrossEntropy(output, target, padcar)`，该函数根据模型的输出 `output`、目标输出 `target` 和填充字符的编码 `padcar` 来计算损失，并忽略填充字符。请注意，在计算时不要使用任何循环。

最后的修改是在生成方面。

### 问题 3
在 `generate.py` 中，实现一个生成函数，以便生成序列直到遇到 EOS 字符（仍然要考虑设定一个最大长度！）。你可以选择在获得的分布中进行随机生成，或者在每个时间步选择最可能的字符进行确定性生成。

在 TP3 中，你使用了对每个字符的独热编码，随后是一个线性模块。Torch 的 `nn.Embedding` 模块允许将这两个步骤结合起来，以避免创建（不必要且耗费资源的）独热编码向量。

### 问题 4

修改你的代码，直接通过 `nn.Embedding` 模块使用嵌入，而不是手动使用独热编码表示。

## 2  考虑长距离依赖：LSTM 和 GRU

循环神经网络（RNN）是一种优先用于处理序列的架构类型，它通过一个函数 $ f $ 迭代地对序列进行编码，从而计算下一个状态：$ h_t = f(x_t, h_{t-1}) $。

RNN 在捕捉长距离依赖关系时存在困难。这是因为当反向传播经过几个时间步时，梯度非常不稳定；简单地说，输入在时间步 $ t + k $ 的贡献将是以下形式（参见 [3]）：

$$
\|W^k\|_2 = \Sigma |\lambda_i|^{2k}
$$

在本次实验中，我们将研究 RNN 的变体，特别是长短期记忆网络（LSTM）和门控循环单元（GRU）（也可以参见这篇文章）。

LSTM（长短期记忆网络）由一个外部状态 $ h_t $（类似于常规 RNN 的状态）和一个内部状态 $ C_t $ 组成，后者代表网络的“长期”记忆。在每个时间步中，$ C_t $ 根据前一个内部状态 $ h_{t-1} $ 和当前输入 $ x_t $ 进行更新，明确指定需要“遗忘”的过去信息和需要“保留”的未来信息。新的外部状态是根据新的内部状态计算得出的。

这种写入机制基于借鉴于逻辑的门控概念，允许屏蔽掉部分不重要的信号。在神经网络的情况下，门控是一个连续函数（而不是逻辑中的离散函数），通常是一个线性层后跟一个 sigmoid 激活函数（因此产生的输出在 0 和 1 之间）。

内部状态和外部状态的演变由以下方程定义（图1），该方程依赖于三个门（遗忘门、输入门和输出门）：

以下是状态更新的公式：

$$
f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) \quad \text{（遗忘门）}
$$

$$
i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i) \quad \text{（输入门）}
$$

$$
C_t = f_t \odot C_{t-1} + i_t \odot \tanh(W_C \cdot [h_{t-1}, x_t] + b_C) \quad \text{（内部记忆更新）}
$$

$$
o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o) \quad \text{（输出门）}
$$

$$
h_t = o_t \odot \tanh(C_t) \quad \text{（输出）}
$$

其中 $[a, b]$ 是向量的连接，$\odot$ 是逐元素乘积（Hadamard 乘积）。

GRU（门控循环单元）是 LSTM 的简化版（还有其他变体），其内部状态和外部状态是相同的，输入门和遗忘门合并为一个门（设定为 $ i_t := 1 - f_t $）。定义 GRU 的方程（图2）为：

$$
z_t = \sigma(W_z \cdot [h_{t-1}, x_t]) \quad \text{（更新门）}
$$

$$
r_t = \sigma(W_r \cdot [h_{t-1}, x_t]) \quad \text{（重置门）}
$$

$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tanh(W \cdot [r_t \odot h_{t-1}, x_t]) \quad \text{（当前状态）}
$$

### 问题 5

实现一个 GRU 和一个 LSTM。请注意，对于 LSTM，有一个内部状态。比较它们在似然性（损失）上的结果，以及生成的序列的质量。

使用 TensorBoard 来监控梯度的大小以及不同门的值的变化。为此，可以使用 `add_histogram`。如果遇到不稳定的问题且梯度的大小很大，可以使用梯度裁剪（gradient clipping）。

## 3. 束搜索（Beam Search）

在您的生成尝试中，您会发现如果在每个时间步都取 argmax，您很少会得到可理解的句子（实际上，您是在进行一个贪婪的最大似然近似）。在初期，您可以在每个时间步从推断出的分布中进行采样，但结果也不会好太多。通常的解决方案是使用束搜索（beam search）来近似整个生成序列的 argmax：束搜索的思路是在任何时候保留 K 条序列及其对应的对数概率；在第 $t+1$ 步中，对于每一条序列 $s$，生成 K 个最可能的符号。然后选择 K 条长度为 $t+1$ 的最可能序列，并重复该过程。

### 问题 6

实现一个束搜索（beam search）用于句子的生成，并对获得的结果进行定性比较。

生成序列的质量可以通过使用采样技术进一步提高，例如**Nucleus Sampling**（核采样），其方法是通过考虑最可能的字符来定义生成的概率，并设置一个阈值 $\alpha$（一个超参数，例如 0.95），仅选择能够最好覆盖这一概率质量的输出。形式上，如果 $I_\alpha(p,s)$ 是满足以下条件的最小符号集合：

$$
\sum_{I_{\alpha }(p,s) } p(y|s)\ge \alpha  
$$

核概率被定义为：

$$
p_{nucleus ~ K}(y|s_{t})=\mathbb{I}    \left [ y\in I_{\alpha } (p,s_{t})\right ] \times \frac{p(y|s_{t})}{ {\textstyle \sum_{y'\in I_{\alpha } (p,s_{t})}p(y'|s_{t})} } 
$$

### 问题 7

使用核采样（Nucleus Sampling）实现束搜索（Beam Search）。